# CATALYST

![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)

PyTorch framework for Deep Learning R&D.

It focuses on reproducibility, rapid experimentation, and codebase reuse so you can create something new rather than write yet another train loop. Break the cycle - use the Catalyst!

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, Subset, DataLoader
from sklearn.model_selection import train_test_split
from catalyst import dl, utils


def get_dataset(csv_file: str, labels: bool = True) -> TensorDataset:
    '''Create tensor dataset from csv file
    '''
    df = pd.read_csv(csv_file)
    if not labels:
        df['label'] = 0
    y = torch.tensor(df.label.values)
    x = torch.tensor(df.drop(['label'], axis=1).values)
    return TensorDataset(x / 255, y)


dataset: TensorDataset = get_dataset('../input/digit-recognizer/train.csv')
test_dataset: TensorDataset = get_dataset('../input/digit-recognizer/test.csv', labels=False)

# Split the dataset
idx: np.array = np.arange(len(dataset))
train_idx, test_idx = train_test_split(idx, train_size=0.9)
train_dataset = Subset(dataset, train_idx, )
valid_dataset = Subset(dataset, test_idx)
print(len(train_dataset), len(valid_dataset), len(test_dataset))

loaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True),
    'valid': DataLoader(valid_dataset, batch_size=32, shuffle=False),
}
test_loader : DataLoader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 10)
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

runner = dl.SupervisedRunner(
    input_key='features', output_key='logits', target_key='targets', loss_key='loss'
)

37800 4200 28000


In [2]:
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    callbacks=[
        dl.AccuracyCallback(input_key='logits', target_key='targets', topk_args=(1, 3, 5)),
        dl.PrecisionRecallF1SupportCallback(input_key='logits', target_key='targets', num_classes=10),
    ],
    logdir='./logs',
    valid_loader='valid',
    valid_metric='loss',
    minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)

1/1 * Epoch (train):   0%|          | 0/1182 [00:00<?, ?it/s]

train (1/1) accuracy01: 0.8784126984126982 | accuracy01/std: 0.07205458941880563 | accuracy03: 0.9731746031746037 | accuracy03/std: 0.04479924490086416 | accuracy05: 0.990476190476191 | accuracy05/std: 0.03046046809731511 | f1/_macro: 0.8767688264439519 | f1/_micro: 0.8784076984411587 | f1/_weighted: 0.8782172953642555 | f1/class_00: 0.9427227573638143 | f1/class_01: 0.9491875030289556 | f1/class_02: 0.8667338953607854 | f1/class_03: 0.8453796154302953 | f1/class_04: 0.8831820358223229 | f1/class_05: 0.8101065260916702 | f1/class_06: 0.925672488675567 | f1/class_07: 0.8991172491936427 | f1/class_08: 0.813591193337072 | f1/class_09: 0.8319950001353946 | loss: 0.46729418902406616 | loss/mean: 0.46729418902406616 | loss/std: 0.30029187102151095 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.8768893695958313 | precision/_micro: 0.8784126984126984 | precision/_weighted: 0.8781443908400692 | precision/class_00: 0.9357482813326282 | precision/class_01: 0.9413140051437924 | precision/class_0

1/1 * Epoch (valid):   0%|          | 0/132 [00:00<?, ?it/s]

valid (1/1) accuracy01: 0.8854761904761903 | accuracy01/std: 0.05333662731240196 | accuracy03: 0.9795238095238099 | accuracy03/std: 0.02434689962940028 | accuracy05: 0.9940476190476192 | accuracy05/std: 0.012864735645996045 | f1/_macro: 0.8832272830580384 | f1/_micro: 0.8854711905044238 | f1/_weighted: 0.8841357029664321 | f1/class_00: 0.9549950011511661 | f1/class_01: 0.9389339482289837 | f1/class_02: 0.8972923004164316 | f1/class_03: 0.8739240463570291 | f1/class_04: 0.9226754189078831 | f1/class_05: 0.8616550845049183 | f1/class_06: 0.9495142318356745 | f1/class_07: 0.8361855676660019 | f1/class_08: 0.824442357980594 | f1/class_09: 0.7726548735317017 | loss: 0.49341805154368984 | loss/mean: 0.49341805154368984 | loss/std: 0.3256284591763327 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.8941136111000864 | precision/_micro: 0.8854761904761905 | precision/_weighted: 0.8935189040197875 | precision/class_00: 0.9695431472081218 | precision/class_01: 0.9001919385796545 | precision/class

In [3]:
# model evaluation
metrics = runner.evaluate_loader(
    loader=loaders['valid'],
    callbacks=[dl.AccuracyCallback(input_key='logits', target_key='targets', topk_args=(1, 3, 5))]
)
assert 'accuracy01' in metrics.keys()

valid (1/1) accuracy01: 0.8854761904761903 | accuracy01/std: 0.05333662731240196 | accuracy03: 0.9795238095238099 | accuracy03/std: 0.02434689962940028 | accuracy05: 0.9940476190476192 | accuracy05/std: 0.012864735645996045
* Epoch (1/1) 


In [4]:
# model inference
submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission.Label = [p['logits'].cpu().numpy().argmax() for p in runner.predict_loader(loader=test_loader)]
submission.to_csv('submission.csv', index=False)